In [ ]:
! pip install transformers torch


     |████████████████████████████████| 4.2 MB 31.8 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 55.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# This cell is to style the Google Colab printed output properly (Just run this)
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
from transformers import pipeline
story_generator = pipeline("text-generation", 
                           "pranavpsv/gpt2-genre-story-generator")


Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/166 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [ ]:
summarizer = pipeline("summarization", "sshleifer/distilbart-cnn-12-6")

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
def get_extra_part(str1, str2):
    extra_portion = str2[len(str1):]

    # begin = len(str1)
    # for idx in range(begin, len(str2)):
    #     extra_portion += str2[idx]
    return extra_portion

get_extra_part("Hi I am", "Hi I am Moumita")

' Moumita'

In [ ]:
def generate_options(no_options, init_genre_id, genre_selection_map, prev_story, prev_seq, new_story_len, genres):
  story_versions = []
  newly_generateds = []
  for i in range(no_options):
    genre = genre_selection_map[genres[init_genre_id]][i]
    prev_story_len = len(prev_story)
    current_max_len = prev_story_len + new_story_len
    next_genre = '<' + genre + '>'
    next_seq = '<BOS> '+ next_genre
    next_prompt = get_extra_part(prev_seq, prev_story) # setting the previous generated story text as the prompt and removing the <BOS> and genre info

    story1 = story_generator(next_seq + next_prompt , max_length = current_max_len) 
    story_text1 = story1[0]["generated_text"]
    newly_generated = get_extra_part(next_seq + next_prompt, story_text1)

    story_versions.append(story_text1)
    newly_generateds.append(newly_generated)

  return story_versions, newly_generateds

In [ ]:
def truncate_text(text):
    while len(text) > 0 and text[len(text) - 1] != '.':
        text = text[:-1]
    return text

def has_multiple_sentences(text):
    idx = 0
    for character in text:
        if character == '.' and idx >= 10:
            return True
        idx += 1
    return False



In [ ]:
def generate_summaries(no_options, newly_generated):
  summaries = []
  for i in range(no_options):
      summarized = summarizer(newly_generated[i], min_length=15, max_length=25)
      summary_text = summarized[0]['summary_text']
      if has_multiple_sentences(summary_text):
          summary_text = truncate_text(summary_text)
      summaries.append(summary_text)

  return summaries

In [ ]:
import random
two_randoms = random.sample(range(1, 5), 2)
print(two_randoms)

[2, 4]


In [ ]:
# Modify this cell to create your own stories

# Input format: <BOS> <genre> Optional prompt...
# Supported genres: superhero, sci_fi, horror, action, thriller, drama
def run(init_genre_id):
    genres = ['superhero', 'action', 'horror', 'thriller', 'sci_fi', 'drama']
    genre_selection_map = {'superhero': ['superhero', 'action', 'thriller', 'sci_fi', 'drama'],
                           'action': ['action', 'superhero', 'horror', 'thriller', 'drama'],
                           'horror': ['horror', 'superhero', 'action', 'thriller', 'drama'],
                           'sci-fi': ['sci-fi', 'superhero', 'horror', 'thriller', 'drama'],
                           'drama': ['drama', 'action', 'thriller', 'sci_fi', 'horror']}
    '''
    User defined parameters
    '''
    init_genre = '<' + genres[init_genre_id] + '>'

    init_seq = '<BOS> '+ init_genre

    init_prompt = ""
    each_story_len = 100

    no_options = len(genre_selection_map[genres[init_genre_id]])

    how_many_randoms = 2

    '''
    '''

    current_max_len = each_story_len
    story = story_generator(init_seq + init_prompt, max_length = current_max_len) 
    story_text = story[0]["generated_text"]
    prev_seq = init_seq
    prev_story_text = story_text

    print("---------------------------------------------------------------")
    print(story_text)
    print()


    story_versions, newly_generateds = generate_options(no_options, init_genre_id, genre_selection_map, prev_story_text, prev_seq, each_story_len, genres)


    for i in range(no_options):
        print("---------------------------------------------------------------")
        print(story_versions[i])
        print()


    # summaries = []
    for i in range(no_options):
        summaries = generate_summaries(no_options, newly_generateds)
        # summarized = summarizer(story_versions[i], min_length=15, max_length=25)
        # summary_text = summarized[0]['summary_text']
        # summaries.append(summary_text)
        print("summary for option", i, " = ", summaries[i])
        print()

    random_numbers = [0]
    random_numbers += random.sample(range(1, no_options), how_many_randoms)

    new_prompts = []
    passages = []
    for i in range(len(random_numbers)):
      next_genre_name = genre_selection_map[genres[init_genre_id]][random_numbers[i]]
      next_genre = '<' + genres[init_genre_id] + '>'
      next_seq = '<BOS>' + next_genre 
      
      new_prompt_phase1 = get_extra_part(prev_seq, prev_story_text) + summaries[random_numbers[i]]

      current_max_len = len(next_seq + new_prompt_phase1) + each_story_len

      story1 = story_generator(next_seq + new_prompt_phase1 , max_length = current_max_len) 
      story_text1 = story1[0]["generated_text"]
      # newly_generated = get_extra_part(next_seq + next_prompt, story_text1)
      new_prompts.append(summaries[random_numbers[i]])
      passages.append(story_text1)
      # story_versions.append(story_text1)
      # newly_generateds.append(newly_generated)

    

    for i in range(len(random_numbers)):
        print("********************************************************")
        print(passages[i])
        print() 
    return story_text, story_versions, summaries, new_prompts, passages
    



In [ ]:
def posprocessing(context, options, passages):
  context_without_tokens = get_extra_part('<BOS>', context)
  context_with_tokens = '<SOS>' + context_without_tokens + '<EOS>'
  options_with_tokens = []
  prompts_with_tokens = []
  passages_with_tokens = []
  
  for i in range(len(options)):
    opt_token = '<SOS>' + options[i] + '<EOS>'
    options_with_tokens.append(opt_token)
    prompt = '<SOS>' + context_without_tokens + '<CLS>' + options[i] + '<EOS>'
    prompts_with_tokens.append(prompt)
    pass_wo_tokens = get_extra_part('<BOS>', passages[i])
    pass_with_tokens = '<SOS>' + pass_wo_tokens + '<EOS>'
    passages_with_tokens.append(pass_with_tokens)


  return context_with_tokens, options_with_tokens, prompts_with_tokens, passages_with_tokens



In [ ]:
def to_csv(id, init_genre, context_with_tokens, options_with_tokens, prompts_with_tokens, passages_with_tokens, bleus):
  data = {}
  data['genre'] = init_genre
  data['Context'] = context_with_tokens
  for i in range(len(options_with_tokens)):
    data['Option' + str(i)] = options_with_tokens[i]
    data['Prompt' + str(i)] = prompts_with_tokens[i]
    data['Passage' + str(i)] = passages_with_tokens[i]
  
  for i in range(len(bleus)):
    data['Bleu' + str(i)] = str(bleus[i])

    
  return data


In [ ]:
import  nltk.translate.bleu_score as bleu

def calculate_bleu(passages):
  reference = passages[0].split()
  candidate1 = passages[1].split()
  candidate2 = passages[2].split()
  bleu1 = bleu.sentence_bleu(reference, candidate1)
  bleu2 = bleu.sentence_bleu(reference, candidate2)
  print("BLEU Score for", 1 , ":", bleu.sentence_bleu(reference, candidate1))
  print("BLEU Score for", 2 , ":", bleu.sentence_bleu(reference, candidate2))

  return bleu1, bleu2




In [ ]:
import pandas as pd
import time

yourname = 'Moumita1'
str_init = yourname + 'genrebased'
genres = ['superhero', 'action', 'horror', 'thriller', 'sci_fi', 'drama']
st_time = time.time()

total_files = 1
for fileNo in range(total_files):
  data_tuple_list = []

  for i in range(len(genres)):
    story_text, story_versions, summaries, new_prompts, passages = run(i)
    bleus = list(calculate_bleu(passages))
    # bleu1, bleu2 = calculate_bleu(passages)
    context_with_tokens, options_with_tokens, prompts_with_tokens, passages_with_tokens = posprocessing(story_text, new_prompts, passages)
    data = to_csv(i, genres[i], context_with_tokens, options_with_tokens, prompts_with_tokens, passages_with_tokens, bleus)
    data_tuple_list.append(data)

  dataframe = pd.DataFrame(data_tuple_list)
  dataframe.to_csv(str_init + 'data' + str(fileNo)+ '.csv')
en_time = time.time()

print("Total time taken = ", en_time - st_time, "seconds")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------
<BOS> <superhero>  Lead-up: Magneto and Storm are shown fighting through the battle sewer with Magneto saying he wants to get help while Storm doesn't want to. Confirmations are made in each of the city and in Galactus' home, but Magneto is revealed to be still fighting with the Omega Men.Magneto begins with "A-Bomb!", but when Storm's bomb misses him, the Omega Men come to him and tell him the fight is over. Magik



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------
<BOS> <superhero>  Lead-up: Magneto and Storm are shown fighting through the battle sewer with Magneto saying he wants to get help while Storm doesn't want to. Confirmations are made in each of the city and in Galactus' home, but Magneto is revealed to be still fighting with the Omega Men.Magneto begins with "A-Bomb!", but when Storm's bomb misses him, the Omega Men come to him and tell him the fight is over. Magik thinks that the Omega Men were just waiting for the opportunity. Magneto, enraged with Storm and the Omega Men, attacks the Omega Men before Magneto stops him. The Omega Men are then attacked by Magik's allies. Magneto has a showdown with them with the help of his own power in a battle that sees Storm defeated.Magik then has a confrontation with The Mole, who has a powerful weapon that transforms magneto into a huge tornado that slices through the city. Magneto then fights with Magneto in a huge arena. The Mole 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


BLEU Score for 1 : 0.25433171791051756
BLEU Score for 2 : 0.3839817133079349


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------
<BOS> <action> The film comes to a close in the second half of the war. As Allied forces advance into Germany, the British command decides to take the last stand during the battle. Colonel Rainscott (Carradine) is given command of the Luftwaffe's 4th Infantry Division, as he has a long-standing reputation for precision. Rainscott becomes a major in front of the Luftwaffe commandant, Major Epping (Kenny Baker), and a



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
